## Importando os dados

In [77]:
# abrir a conexão com o banco de dados
import sqlite3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
conn = sqlite3.connect("database.db")

In [13]:
# consulta os dados no banco de dados
consulta_atividade = """

    SELECT
     *
    FROM flight_activity fa LEFT JOIN flight_loyalty_history fh ON (fa.loyalty_number = fh.loyalty_number)
    """
df_atividade = pd.read_sql_query(consulta_atividade, conn)

In [14]:
df_atividade.head()

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,...,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,...,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,...,College,NaN,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,...,College,NaN,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN
3,100214,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,63253.0,Married,Star,4170.57,Standard,2015,8,NaN,NaN
4,100272,2017,1,0,0,0,0,0.0,0,0,...,Bachelor,91163.0,Divorced,Star,6622.05,Standard,2014,1,NaN,NaN


## JOIN

In [ ]:
consulta_atividade = """
  SELECT
    fh.loyalty_number,
    fh.country,
    fh.city,
    fa.year,
    fa.month,
    fa.flights_booked

  FROM
    flight_loyalty_history fh JOIN flight_activity fa ON (fh.loyalty_number = fa.loyalty_number)


"""
df_atividade = pd.read_sql_query(consulta_atividade, conn)
df_atividade.head()

In [ ]:
df_atividade.info()

In [ ]:
df_atividade.shape

## Exercícios da Aula 02

In [ ]:
# EXERCÍCIO 1
# selecionar o número do cartão de fidelidade, a cidade e o
# gênero dos passageiros que tem o cartão Star mas nunca realizaram
# nenhuma viagem
consulta_atividade = """

     SELECT
      fa.loyalty_number,
      fa.distance,
      fh.city,
      fh.gender,
      fh.loyalty_card
     FROM
      flight_activity fa LEFT JOIN flight_loyalty_history fh ON (fh.loyalty_number = fa.loyalty_number)
     WHERE
        loyalty_card == 'Star' AND distance == 0
     """
df_atividade = pd.read_sql_query(consulta_atividade, conn)
df_atividade

In [ ]:
# EXERCÌCIO 2
# selecionar os números do cartão de fidelidade, o gênero e a cidade de todos os
# passageiros do sexo feminino que moram na cidade de Toronto, fizeram mais de 30
# viagens no total e tem o cartão de fidelidade do tipo Aurora.
consulta_atividade = """

     SELECT
      fa.loyalty_number,
      fh.gender,
      fh.city,
      fa.total_flights,
      fh.loyalty_card
     FROM
      flight_activity fa LEFT JOIN flight_loyalty_history fh
      ON (fh.loyalty_number = fa.loyalty_number)
     WHERE
        gender == 'Female'
        AND city == 'Toronto'
        AND total_flights == 30
        AND loyalty_card == 'Aurora'
     """
df_atividade = pd.read_sql_query(consulta_atividade, conn)
df_atividade

In [ ]:
# EXERCÌCIO 3
# Selecionar os números do cartão de fidelidade, o tipo do cartão, o gênero e os
# pontos acumulados, dos passageiros com salário acima de 13200, estado civil como
# casado e nível acadêmico como mestrado e número de voos agendados igual ao
# número total de voos.
consulta_atividade = """

     SELECT
      fa.loyalty_number,
      fh.loyalty_card,
      fh.gender,
      fa.points_accumulated

     FROM
      flight_activity fa LEFT JOIN flight_loyalty_history fh
      ON (fh.loyalty_number = fa.loyalty_number)
     WHERE
        salary > 13200
        AND marital_status == 'Married'
        AND education == 'Master'
        AND flights_booked = total_flights
     """
df_atividade = pd.read_sql_query(consulta_atividade, conn)
df_atividade

In [ ]:
# EXERCÌCIO 4
# Qual o valor da soma total da distância percorrida pelos voos registrados na
# planilha de dados?

df_atividade.loc[:, 'distance'].sum()

490350765

In [ ]:
# EXERCÌCIO 5
# Qual o salário médio dos passageiros?
consulta_atividade = "SELECT * FROM flight_loyalty_history"
df_atividade = pd.read_sql_query(consulta_atividade, conn)
df_atividade.loc[:, 'salary'].mean()

79245.6094087527

In [ ]:
# EXERCÌCIO 6
# Qual o valor total de pontos acumulados?
consulta_atividade = "SELECT * FROM flight_activity"
df_atividade = pd.read_sql_query(consulta_atividade, conn)
df_atividade.loc[:, 'points_accumulated'].sum()

50172736.25

## Preparação dos dados

In [ ]:
# Verifica o número de dados faltantes em cada coluna
df_atividade.isna().sum()

In [ ]:
df_atividade.info()

In [24]:
# Selecionar somente as colunas numéricas
colunas = ["year","month","flights_booked", "flights_with_companions", "total_flights",
           "distance", "points_accumulated", "salary", "clv", "loyalty_card"]
df_colunas_numericas = df_atividade.loc[:, colunas]

In [ ]:
# Verifica o número de dados faltantes nas colunas numéricas
df_colunas_numericas.isna().sum()

In [29]:
# Remover as linhas que contém dados faltantes
df_dados_completos = df_colunas_numericas.dropna()

In [ ]:
# Favor nova verificação nos dados completos
df_dados_completos.isna().sum()

In [49]:
df_dados_completos.shape

(302952, 10)

## Machile Learning

In [36]:
from sklearn import tree as tr

# Definição do algoritmo
modelo = tr.DecisionTreeClassifier(max_depth=2)

In [47]:
# Treinamento do algoritmo
X_atributos = df_dados_completos.drop(columns="loyalty_card")
y_rotulos = df_dados_completos.loc[:, "loyalty_card"]
modelo_treinado = modelo.fit(X_atributos, y_rotulos)

In [ ]:
X_atributos

In [ ]:
y_rotulos

In [ ]:
tr.plot_tree(modelo_treinado, filled=True)

## Apresentando o resultado

In [81]:
X_novo = X_atributos.sample()
previsao = (modelo_treinado.predict_proba(X_novo)) * 100

print(f"Aurora: {previsao[0][0]:.2f}% - Nova: {previsao[0][1]:.2f}% -  Start: {previsao[0][2]:.2f}%")

Aurora: 0.00% - Nova: 51.25% -  Start: 48.75%


## Painel de Visualização

In [ ]:
!pip install gradio

In [85]:
import gradio as gr

In [88]:
with gr.Blocks() as demo:
  # Título do painel
  gr.Markdown(""" # Propensao de compra """)

  with gr.Row():
    with gr.Column():
      gr.Markdown(""" # Coluna 1 """)

      with gr.Row():
        gr.Markdown(""" # Linha 1 """)

    with gr.Column():
      gr.Markdown(""" # Coluna 2 """)
  # Botao predict
  # predict_btn.click()

demo.launch(debug=True, share=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
